In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
p = '/home/dhasade/audits/fairness-audits-with-collaboration/data/propublica/compas-scores-two-years.csv'

In [3]:
df_o = pd.read_csv(p)

In [4]:
df_o

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


In [5]:
df_o.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [6]:
df = df_o[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
               'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid',
               'c_jail_in', 'c_jail_out']]

# Applying the filters as in the R code
df = df[df['days_b_screening_arrest'] <= 30]
df = df[df['days_b_screening_arrest'] >= -30]
df = df[df['is_recid'] != -1]
df = df[df['c_charge_degree'] != "O"]
df = df[df['score_text'] != 'N/A']

# Getting the number of rows after filtering
num_rows = df.shape[0]

print(num_rows)

6172


In [7]:
df

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,23,F,African-American,Less than 25,Medium,Male,0,-1.0,7,0,0,2013-11-22 05:18:27,2013-11-24 02:59:20
7210,23,F,African-American,Less than 25,Low,Male,0,-1.0,3,0,0,2014-01-31 07:13:54,2014-02-02 04:03:52
7211,57,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,2014-01-13 05:48:01,2014-01-14 07:49:46
7212,33,M,African-American,25 - 45,Low,Female,3,-1.0,2,0,0,2014-03-08 08:06:02,2014-03-09 12:18:04


In [8]:
len(df_o), len(df)

(7214, 6172)

In [9]:
# keep rows in df_o whose index is in df
df_o = df_o.loc[df.index]

In [10]:
df_o

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


In [18]:
# create a new final df
new_df = pd.DataFrame()
# column 1: African_American

new_df['African_American'] = np.where(df_o['race'] == 'African-American', 1, 0)

In [23]:
print(len(new_df))
new_df.head()

6172


,African_American
0,0
1,1
2,1
3,0
4,0


In [24]:
# column 2: Female

new_df['Female'] = np.where(df_o['sex'] == 'Female', 1, 0)
print(len(new_df))

new_df.head()

6172


,African_American,Female
0,0,0
1,1,0
2,1,0
3,0,0
4,0,0


In [25]:
# column 3: Age_Below_TwentyFive

new_df['Age_Below_TwentyFive'] = np.where(df_o['age'] < 25, 1, 0)
print(len(new_df))
new_df.head()

6172


,African_American,Female,Age_Below_TwentyFive
0,0,0,0
1,1,0,0
2,1,0,1
3,0,0,0
4,0,0,0


In [26]:
# column 4: Two_yr_Recidivism

new_df['Two_yr_Recidivism'] = np.where(df_o['two_year_recid'] == 1, 1, 0)
print(len(new_df))
new_df.head()

6172


,African_American,Female,Age_Below_TwentyFive,Two_yr_Recidivism
0,0,0,0,0
1,1,0,0,1
2,1,0,1,1
3,0,0,0,0
4,0,0,0,1


In [31]:
# column 5: Number_of_Priors

new_df['Number_of_Priors'] = df_o['priors_count'].values
print(len(new_df))
new_df.head()

6172


,African_American,Female,Age_Below_TwentyFive,Two_yr_Recidivism,Number_of_Priors
0,0,0,0,0,0
1,1,0,0,1,0
2,1,0,1,1,4
3,0,0,0,0,0
4,0,0,0,1,14


In [44]:
# column 6: Misdemeanor

new_df['Misdemeanor'] = np.where(df_o['c_charge_degree'] == 'M', 1, 0)
print(len(new_df))
new_df.head()

6172


,African_American,Female,Age_Below_TwentyFive,Two_yr_Recidivism,Number_of_Priors,Misdemeanor
0,0,0,0,0,0,0
1,1,0,0,1,0,0
2,1,0,1,1,4,0
3,0,0,0,0,0,1
4,0,0,0,1,14,0


In [33]:
# compare to the original csv
p_avant = '/home/dhasade/audits/ml-audits/data/propublica/propublica_raw.csv'
df_avant = pd.read_csv(p_avant)

In [45]:
print(df_avant['Misdemeanor'].value_counts())
print(new_df['Misdemeanor'].value_counts())

Misdemeanor
0    3970
1    2202
Name: count, dtype: int64
Misdemeanor
0    3970
1    2202
Name: count, dtype: int64


In [ ]:
print(df_avant['Number_of_Priors'].value_counts())
print(new_df['Number_of_Priors'].value_counts())

In [37]:
print(df_avant['Two_yr_Recidivism'].value_counts())
print(new_df['Two_yr_Recidivism'].value_counts())

Two_yr_Recidivism
0    3363
1    2809
Name: count, dtype: int64
Two_yr_Recidivism
0    3363
1    2809
Name: count, dtype: int64


In [38]:
print(df_avant['Age_Below_TwentyFive'].value_counts())
print(new_df['Age_Below_TwentyFive'].value_counts())

Age_Below_TwentyFive
0    4825
1    1347
Name: count, dtype: int64
Age_Below_TwentyFive
0    4825
1    1347
Name: count, dtype: int64


In [42]:
print(df_avant['African_American'].value_counts())
print(new_df['African_American'].value_counts())

African_American
1    3175
0    2997
Name: count, dtype: int64
African_American
1    3175
0    2997
Name: count, dtype: int64


In [43]:
print(df_avant['Female'].value_counts())
print(new_df['Female'].value_counts())

Female
0    4997
1    1175
Name: count, dtype: int64
Female
0    4997
1    1175
Name: count, dtype: int64


## Final check to see if matching columns have same values

In [48]:
features_avant = '/home/dhasade/audits/ml-audits/data/propublica/features.csv'
df_features_avant = pd.read_csv(features_avant)

features_apres = '/home/dhasade/audits/fairness-audits-with-collaboration/data/propublica/features.csv'
df_features_apres = pd.read_csv(features_apres)

In [50]:
# check if the following columns have same values
cols = ["Female", "African_American", "Age_Below_TwentyFive", "Misdemeanor", "Number_of_Priors"]

# check if the columns have the same values
(df_features_avant[cols] == df_features_apres[cols]).all()


Female                  True
African_American        True
Age_Below_TwentyFive    True
Misdemeanor             True
Number_of_Priors        True
dtype: bool

In [51]:
df_features_avant[cols].head()

,Female,African_American,Age_Below_TwentyFive,Misdemeanor,Number_of_Priors
0,0,0,0,0,0
1,0,1,0,0,0
2,0,1,1,0,1
3,0,0,0,1,0
4,0,0,0,0,1


In [52]:
df_features_apres[cols].head()

,Female,African_American,Age_Below_TwentyFive,Misdemeanor,Number_of_Priors
0,0,0,0,0,0
1,0,1,0,0,0
2,0,1,1,0,1
3,0,0,0,1,0
4,0,0,0,0,1


In [53]:
# count where the values are different
(df_features_avant[cols] != df_features_apres[cols]).sum()

Female                  0
African_American        0
Age_Below_TwentyFive    0
Misdemeanor             0
Number_of_Priors        0
dtype: int64

In [54]:
labels_avant = '/home/dhasade/audits/ml-audits/data/propublica/labels.csv'
df_labels_avant = pd.read_csv(labels_avant)

labels_apres = '/home/dhasade/audits/fairness-audits-with-collaboration/data/propublica/labels.csv'
df_labels_apres = pd.read_csv(labels_apres)

In [55]:
# check if the following columns have same values
cols = ["Y"]

# check if the columns have the same values
(df_labels_avant[cols] == df_labels_apres[cols]).all()

Y    True
dtype: bool

In [56]:
df_labels_avant[cols].head()

,Y
0,0
1,1
2,1
3,0
4,1


In [58]:
df_labels_apres[cols].head()

,Y
0,0
1,1
2,1
3,0
4,1
